In [12]:
import os

In [13]:
%pwd

'c:\\Users\\lpaes\\projects\\Xray_img_detection'

In [14]:
#os.chdir('../')
%pwd

'c:\\Users\\lpaes\\projects\\Xray_img_detection'

In [15]:
import tensorflow as tf

In [16]:
#model = tf.keras.models.load_model('artifacts/training/model.h5')

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    validation_data: Path
    metric_file_name: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    mlflow_uri: str
    artifact_uri : Path
    mlflow_experiment: Path

In [18]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    os.environ['MLFLOW_TRACKING_URI']=
    os.environ['MLFLOW_TRACKING_USERNAME']=
    os.environ['MLFLOW_TRACKING_PASSWORD']=

    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root, self.config.evaluation.root_dir])

    def get_validation_config(self) -> EvaluationConfig:
        config = self.config
        eval_config = EvaluationConfig(
            path_of_model = Path(config.training.trained_model_path),
            training_data = Path(config.data_ingestion.train_dir),
            validation_data = Path(config.data_ingestion.test_dir),
            metric_file_name = Path(config.evaluation.metric_file_name),
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
            mlflow_uri=os.environ['MLFLOW_TRACKING_URI'],
            artifact_uri = Path(config.evaluation.artifact_uri),
            mlflow_experiment = Path(config.evaluation.mlflow_experiment),
                       )
        return eval_config

In [19]:
from urllib.parse import urlparse
import mlflow

class Evaluation:
    def __init__(self, config:EvaluationConfig):
        self.config = config 
    
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            #validation_split = 0.20
        )

        dataflow_kwargs = dict(target_size = self.config.params_image_size[:-1],
                               batch_size = self.config.params_batch_size,
                               interpolation = 'bilinear',
                               class_mode='categorical',
                              ) 

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            #subset='validation' # This one should turn on if the data origen for test and val are from the same folder
            shuffle=False,
            **dataflow_kwargs,
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        model = self.load_model(path=self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
    
    def log_into_mlflow(self):
        model = self.load_model(path=self.config.path_of_model)

        datagenerator_kwargs = dict(
            rescale = 1./255,
            #validation_split = 0.20
        )

        dataflow_kwargs = dict(target_size = self.config.params_image_size[:-1],
                               batch_size = self.config.params_batch_size,
                               interpolation = 'bilinear',
                               class_mode='categorical',
                              ) 

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )


        mlflow.set_tracking_uri(self.config.mlflow_uri)  # Set the MLflow tracking URI
        #mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        #mlflow.set_experiment('mlflow_test')
        with mlflow.start_run():

            self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            #subset='validation' # This one should turn on if the data origen for test and val are from the same folder
            shuffle=False,
            **dataflow_kwargs,
            )
            
            # Log metrics
            (loss, accuracy) = model.evaluate(self.valid_generator)
            mlflow.log_metric('loss', loss)
            mlflow.log_metric('accuracy', accuracy)

            # Log parameters
            params_dict = self.config.all_params.to_dict()
            mlflow.log_params(params_dict) # If using ConfigBox, convert it to a dictionary
            

            #Model registry does not work file store
            if tracking_url_type_store != False:
                #Register model
                #There are other ways to use Model Registry, which depends on the use case
                #please refer to the doc for more info:
                #https://mlflow.org/docs/latest/model-registry.html#api-mlflow
                mlflow.tensorflow.log_model(model, artifact_path="model")


    
    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy':self.score[1]}
        #save_json(path=Path('scores.json'), data=scores)
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [20]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    # evaluation.evaluation()
    # evaluation.save_score()
    evaluation.log_into_mlflow()
except Exception as e:
   raise e

[2023-08-03 15:47:07,561: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-03 15:47:07,571: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-03 15:47:07,572: INFO: common: created directory at: artifacts]
[2023-08-03 15:47:07,574: INFO: common: created directory at: artifacts/evaluation]


[2023-08-03 15:47:08,166: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'Foi forçado o cancelamento de uma conexão existente pelo host remoto', None, 10054, None))': /Laidson/Xray_img_detection.mlflow/api/2.0/mlflow/runs/create]
Found 624 images belonging to 2 classes.
39/39 [==============================] - 124s 3s/step - loss: 3.3939 - accuracy: 0.7147


2023/08/03 15:49:14 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\lpaes\AppData\Local\Temp\tmp5zr3j0rx\model\data\model\assets
[2023-08-03 15:49:15,816: INFO: builder_impl: Assets written to: C:\Users\lpaes\AppData\Local\Temp\tmp5zr3j0rx\model\data\model\assets]

